In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
constant_patch = '11.23.1'

# 라이엇 개발자 페이지에서 최신 버전 한국 서버 챔피언 정보 가져오기
champion_info = requests.get("http://ddragon.leagueoflegends.com/cdn/" + constant_patch + "/data/ko_KR/champion.json").json()     

In [3]:
# 챔피언 정보 => DataFrame으로 변환
# champion_df는 'key'로 id값, 'name'으로 챔피언 한국어 이름을 가지는 156rows짜리 DataFrame
champion_df = pd.DataFrame(champion_info['data']).T[['key', 'name']]

# champion_df의 'key' column string => numeric 변환
champion_df['key'] = pd.to_numeric(champion_df['key'])

In [4]:
df = pd.read_csv('11.23.csv', index_col=0)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
df = df.merge(champion_df, how = 'left', left_on = 'championId', right_on = 'key')

In [6]:
data = pd.DataFrame()
data = df[['tierRank', 'position', 'name', 'result']]

In [7]:
# 결측치 드랍
data = data.dropna(axis=0)
# 다시하기 드랍
data.drop(data.loc[data['result']=='UNKNOWN'].index, inplace=True)
print('data : ', len(data))

data :  5065150


In [8]:
# result 숫자로 변환
data.loc[data['result'] == 'WIN', 'result'] = 1
data.loc[data['result'] == 'LOSE', 'result'] = 0
data['games'] = 1
data = data.astype({'result' : 'int'})
# result -> wins로 컬럼명 변경
data.rename(columns = {'result':'wins'},inplace=True)

In [10]:
# 티어별 데이터 프레임 생성

IRON = data[data['tierRank'].str.contains('I')]
print('IRON : ', len(IRON))
BRONZE = data[data['tierRank'].str.contains('B')]
print('BRONZE : ', len(BRONZE))
SILVER = data[data['tierRank'].str.contains('S')]
print('SILVER : ', len(SILVER))
GOLD = data[data['tierRank'].str.contains('G')]
print('GOLD : ', len(GOLD))
PLATINUM = data[data['tierRank'].str.contains('P')]
print('PLATINUM : ', len(PLATINUM))
DIAMOND = data[data['tierRank'].str.contains('D')]
print('DIAMOND : ', len(DIAMOND))
m_list = ['M', 'R', 'C']
m_list = '|'.join(m_list)
ABOVE_MASTER = data[data['tierRank'].str.contains(m_list)]
print('MASTER+ : ', len(ABOVE_MASTER))
# GRAND_MASTER = data[data['tierRank'].str.contains('R')]
# print('GRAND_MASTER : ', len(GRAND_MASTER))
# CHALLENGER = data[data['tierRank'].str.contains('C')]
# print('CHALLENGER : ', len(CHALLENGER))
print('SUM : ', len(IRON)+len(BRONZE)+len(SILVER)+len(GOLD)+len(PLATINUM)+len(DIAMOND)+len(ABOVE_MASTER))

IRON :  54392
BRONZE :  664083
SILVER :  1749375
GOLD :  1776109
PLATINUM :  658328
DIAMOND :  133088
MASTER+ :  29775
SUM :  5065150


In [11]:
m_list

'M|R|C'

In [12]:
IRON_total = IRON.groupby(['position', 'name'], as_index=False).sum()
IRON_sum = IRON.groupby(['position'], as_index=False).sum()
IRON_sum.columns = ['position', 'wins', 'total_games']
IRON_sum = IRON_sum[['position', 'total_games']]
IRON_total = IRON_total.merge(IRON_sum, how = 'left', on = 'position')
IRON_total = IRON_total[IRON_total.games > 50]
IRON_total['winRate'] = IRON_total['wins'] / IRON_total['games']
IRON_total['pickRate'] = IRON_total['games'] / IRON_total['total_games']
IRON_total = IRON_total.sort_values(by=['position', 'winRate', 'pickRate'], ascending=False) 

BRONZE_total = BRONZE.groupby(['position', 'name'], as_index=False).sum()
BRONZE_sum = BRONZE.groupby(['position'], as_index=False).sum()
BRONZE_sum.columns = ['position', 'wins', 'total_games']
BRONZE_sum = BRONZE_sum[['position', 'total_games']]
BRONZE_total = BRONZE_total.merge(BRONZE_sum, how = 'left', on = 'position')
BRONZE_total = BRONZE_total[BRONZE_total.games > 50]
BRONZE_total['winRate'] = BRONZE_total['wins'] / BRONZE_total['games']
BRONZE_total['pickRate'] = BRONZE_total['games'] / BRONZE_total['total_games']
BRONZE_total = BRONZE_total.sort_values(by=['position', 'winRate', 'pickRate'], ascending=False) 

SILVER_total = SILVER.groupby(['position', 'name'], as_index=False).sum()
SILVER_sum = SILVER.groupby(['position'], as_index=False).sum()
SILVER_sum.columns = ['position', 'wins', 'total_games']
SILVER_sum = SILVER_sum[['position', 'total_games']]
SILVER_total = SILVER_total.merge(SILVER_sum, how = 'left', on = 'position')
SILVER_total = SILVER_total[SILVER_total.games > 50]
SILVER_total['winRate'] = SILVER_total['wins'] / SILVER_total['games']
SILVER_total['pickRate'] = SILVER_total['games'] / SILVER_total['total_games']
SILVER_total = SILVER_total.sort_values(by=['position', 'winRate', 'pickRate'], ascending=False) 

GOLD_total = GOLD.groupby(['position', 'name'], as_index=False).sum()
GOLD_sum = GOLD.groupby(['position'], as_index=False).sum()
GOLD_sum.columns = ['position', 'wins', 'total_games']
GOLD_sum = GOLD_sum[['position', 'total_games']]
GOLD_total = GOLD_total.merge(GOLD_sum, how = 'left', on = 'position')
GOLD_total = GOLD_total[GOLD_total.games > 50]
GOLD_total['winRate'] = GOLD_total['wins'] / GOLD_total['games']
GOLD_total['pickRate'] = GOLD_total['games'] / GOLD_total['total_games']
GOLD_total = GOLD_total.sort_values(by=['position', 'winRate', 'pickRate'], ascending=False) 

PLATINUM_total = PLATINUM.groupby(['position', 'name'], as_index=False).sum()
PLATINUM_sum = PLATINUM.groupby(['position'], as_index=False).sum()
PLATINUM_sum.columns = ['position', 'wins', 'total_games']
PLATINUM_sum = PLATINUM_sum[['position', 'total_games']]
PLATINUM_total = PLATINUM_total.merge(PLATINUM_sum, how = 'left', on = 'position')
PLATINUM_total = PLATINUM_total[PLATINUM_total.games > 50]
PLATINUM_total['winRate'] = PLATINUM_total['wins'] / PLATINUM_total['games']
PLATINUM_total['pickRate'] = PLATINUM_total['games'] / PLATINUM_total['total_games']
PLATINUM_total = PLATINUM_total.sort_values(by=['position', 'winRate', 'pickRate'], ascending=False) 

DIAMOND_total = DIAMOND.groupby(['position', 'name'], as_index=False).sum()
DIAMOND_sum = DIAMOND.groupby(['position'], as_index=False).sum()
DIAMOND_sum.columns = ['position', 'wins', 'total_games']
DIAMOND_sum = DIAMOND_sum[['position', 'total_games']]
DIAMOND_total = DIAMOND_total.merge(DIAMOND_sum, how = 'left', on = 'position')
DIAMOND_total = DIAMOND_total[DIAMOND_total.games > 50]
DIAMOND_total['winRate'] = DIAMOND_total['wins'] / DIAMOND_total['games']
DIAMOND_total['pickRate'] = DIAMOND_total['games'] / DIAMOND_total['total_games']
DIAMOND_total = DIAMOND_total.sort_values(by=['position', 'winRate', 'pickRate'], ascending=False) 

ABOVE_MASTER_total = ABOVE_MASTER.groupby(['position', 'name'], as_index=False).sum()
ABOVE_MASTER_sum = ABOVE_MASTER.groupby(['position'], as_index=False).sum()
ABOVE_MASTER_sum.columns = ['position', 'wins', 'total_games']
ABOVE_MASTER_sum = ABOVE_MASTER_sum[['position', 'total_games']]
ABOVE_MASTER_total = ABOVE_MASTER_total.merge(ABOVE_MASTER_sum, how = 'left', on = 'position')
ABOVE_MASTER_total = ABOVE_MASTER_total[ABOVE_MASTER_total.games > 50]
ABOVE_MASTER_total['winRate'] = ABOVE_MASTER_total['wins'] / ABOVE_MASTER_total['games']
ABOVE_MASTER_total['pickRate'] = ABOVE_MASTER_total['games'] / ABOVE_MASTER_total['total_games']
ABOVE_MASTER_total = ABOVE_MASTER_total.sort_values(by=['position', 'winRate', 'pickRate'], ascending=False) 

In [11]:
IRON_total

,position,name,wins,games,total_games,winRate,pickRate
598,T,문도 박사,132,222,10870,0.594595,0.020423
624,T,쉔,41,71,10870,0.577465,0.006532
666,T,잭스,95,167,10870,0.568862,0.015363
685,T,케넨,75,133,10870,0.563910,0.012236
616,T,사이온,33,59,10870,0.559322,0.005428
...,...,...,...,...,...,...,...
87,A,자야,112,226,9181,0.495575,0.024616
73,A,야스오,108,219,9181,0.493151,0.023854
121,A,트위치,33,71,9181,0.464789,0.007733
13,A,드레이븐,35,76,9181,0.460526,0.008278


In [12]:
BRONZE_total

,position,name,wins,games,total_games,winRate,pickRate
690,T,소라카,37,59,131587,0.627119,0.000448
647,T,레오나,45,72,131587,0.625000,0.000547
656,T,리산드라,34,56,131587,0.607143,0.000426
778,T,피즈,38,66,131587,0.575758,0.000502
660,T,말자하,101,176,131587,0.573864,0.001338
...,...,...,...,...,...,...,...
33,A,마스터 이,29,72,128720,0.402778,0.000559
132,A,퀸,21,54,128720,0.388889,0.000420
57,A,사일러스,40,104,128720,0.384615,0.000808
14,A,다리우스,21,55,128720,0.381818,0.000427


In [13]:
SILVER_total

,position,name,wins,games,total_games,winRate,pickRate
700,T,아리,98,171,352458,0.573099,0.000485
718,T,오리아나,61,107,352458,0.570093,0.000304
694,T,스카너,64,114,352458,0.561404,0.000323
761,T,클레드,1150,2060,352458,0.558252,0.005845
711,T,애니비아,96,172,352458,0.558140,0.000488
...,...,...,...,...,...,...,...
28,A,리 신,23,63,349935,0.365079,0.000180
143,A,트린다미어,20,60,349935,0.333333,0.000171
32,A,마스터 이,39,122,349935,0.319672,0.000349
78,A,아지르,16,52,349935,0.307692,0.000149


In [14]:
GOLD_total

,position,name,wins,games,total_games,winRate,pickRate
712,T,애니,108,181,355480,0.596685,0.000509
761,T,코르키,108,191,355480,0.565445,0.000537
742,T,직스,71,126,355480,0.563492,0.000354
666,T,문도 박사,1149,2094,355480,0.548711,0.005891
696,T,스카너,102,186,355480,0.548387,0.000523
...,...,...,...,...,...,...,...
113,A,조이,18,51,358993,0.352941,0.000142
14,A,다리우스,23,68,358993,0.338235,0.000189
28,A,르블랑,21,64,358993,0.328125,0.000178
29,A,리 신,17,59,358993,0.288136,0.000164


In [15]:
PLATINUM_total

,position,name,wins,games,total_games,winRate,pickRate
614,T,노틸러스,47,77,131519,0.610390,0.000585
650,T,베이가,40,69,131519,0.579710,0.000525
624,T,람머스,51,88,131519,0.579545,0.000669
702,T,요릭,397,702,131519,0.565527,0.005338
718,T,제라스,52,92,131519,0.565217,0.000700
...,...,...,...,...,...,...,...
143,A,티모,23,53,133630,0.433962,0.000397
4,A,그레이브즈,30,72,133630,0.416667,0.000539
48,A,블라디미르,23,56,133630,0.410714,0.000419
96,A,이렐리아,58,170,133630,0.341176,0.001272


In [16]:
DIAMOND_total

,position,name,wins,games,total_games,winRate,pickRate
589,T,신지드,96,167,26224,0.574850,0.006368
583,T,쉔,233,408,26224,0.571078,0.015558
637,T,카시오페아,96,169,26224,0.568047,0.006444
597,T,아크샨,50,90,26224,0.555556,0.003432
611,T,요릭,48,87,26224,0.551724,0.003318
...,...,...,...,...,...,...,...
95,A,카이사,637,1354,26952,0.470458,0.050237
112,A,트리스타나,123,262,26952,0.469466,0.009721
20,A,루시안,874,1877,26952,0.465637,0.069642
97,A,칼리스타,132,289,26952,0.456747,0.010723


In [17]:
ABOVE_MASTER_Top = ABOVE_MASTER_total[ABOVE_MASTER_total['position'] == 'T'].sort_values(by=['pickRate'], ascending=False).head(20)

In [18]:
ABOVE_MASTER_total[ABOVE_MASTER_total['position'] == 'T'].sort_values(by=['pickRate'], ascending=False).head(20)

,position,name,wins,games,total_games,winRate,pickRate
433,T,제이스,290,595,5883,0.487395,0.101139
440,T,카밀,262,545,5883,0.480734,0.092640
430,T,잭스,199,356,5883,0.558989,0.060513
362,T,레넥톤,164,330,5883,0.496970,0.056094
423,T,이렐리아,142,268,5883,0.529851,0.045555
347,T,그웬,111,258,5883,0.430233,0.043855
346,T,그레이브즈,113,245,5883,0.461224,0.041645
446,T,케넨,125,234,5883,0.534188,0.039776
463,T,피오라,102,203,5883,0.502463,0.034506
419,T,요네,78,166,5883,0.469880,0.028217


In [19]:
DIAMOND_total[DIAMOND_total['position'] == 'A'].sort_values(by=['winRate'], ascending=False)

,position,name,wins,games,total_games,winRate,pickRate
40,A,벨코즈,35,62,26952,0.564516,0.002300
13,A,드레이븐,247,454,26952,0.544053,0.016845
69,A,야스오,119,221,26952,0.538462,0.008200
54,A,스웨인,60,112,26952,0.535714,0.004156
121,A,하이머딩거,32,60,26952,0.533333,0.002226
35,A,바루스,91,171,26952,0.532164,0.006345
101,A,코그모,103,197,26952,0.522843,0.007309
88,A,징크스,1591,3050,26952,0.521639,0.113164
38,A,베인,636,1223,26952,0.520033,0.045377
85,A,직스,261,505,26952,0.516832,0.018737


In [20]:
IT = IRON_total[IRON_total['position'] == 'T'].sort_values(by=['games'], ascending=True)

In [21]:
champ_stat = pd.read_csv('1124ver_championStats.csv', index_col=0)

In [22]:
champ_stat

,championName,versionIndex,version,hp,hpperlevel,mp,mpperlevel,movespeed,armor,armorperlevel,...,attackspeed,hp_18,mp_18,armor_18,spellblock_18,hpregen_18,mpregen_18,crit_18,attackdamage_18,attackspeed_18
championId,,,,,,,,,,,,,,,,,,,,,
1,Annie,11.23,11.23.409.111,524.0,88,418.0,25.0,335,19,4.00,...,0.579,2020.0,843.0,87.00,38.50,14.85,21.60,0,94.71,0.7129
2,Olaf,11.23,11.23.409.111,575.0,100,316.0,42.0,350,35,3.00,...,0.694,2275.0,1030.0,86.00,53.25,23.80,17.70,0,127.50,1.0125
3,Galio,11.23,11.23.409.111,562.0,112,500.0,40.0,335,24,3.50,...,0.625,2466.0,1180.0,83.50,53.25,21.60,21.40,0,118.50,0.7844
4,TwistedFate,11.23,11.23.409.111,534.0,94,333.0,39.0,330,21,3.15,...,0.651,2132.0,996.0,74.55,38.50,15.70,21.60,0,108.10,1.0074
5,XinZhao,11.23,11.23.409.111,570.0,92,274.0,55.0,345,35,3.50,...,0.645,2134.0,1209.0,94.50,53.25,19.90,14.91,0,114.00,1.0288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,Vex,11.23,11.23.409.111,520.0,90,490.0,32.0,335,23,3.25,...,0.669,2050.0,1034.0,78.25,36.50,16.70,19.60,0,100.75,0.7827
777,Yone,11.23,11.23.409.111,550.0,85,500.0,0.0,345,28,3.40,...,0.625,1995.0,500.0,85.80,53.25,20.25,0.00,0,94.00,0.9969
875,Sett,11.23,11.23.409.111,600.0,93,0.0,0.0,340,33,4.00,...,0.625,2181.0,0.0,101.00,53.25,15.50,0.00,0,128.00,0.8109


In [23]:
champion_df.rename(columns = {'key':'championId'},inplace=True)
champ_stat = champ_stat.merge(champion_df, how = 'left', on = 'championId')

In [24]:
champ_stat

,championId,championName,versionIndex,version,hp,hpperlevel,mp,mpperlevel,movespeed,armor,...,hp_18,mp_18,armor_18,spellblock_18,hpregen_18,mpregen_18,crit_18,attackdamage_18,attackspeed_18,name
0,1,Annie,11.23,11.23.409.111,524.0,88,418.0,25.0,335,19,...,2020.0,843.0,87.00,38.50,14.85,21.60,0,94.71,0.7129,애니
1,2,Olaf,11.23,11.23.409.111,575.0,100,316.0,42.0,350,35,...,2275.0,1030.0,86.00,53.25,23.80,17.70,0,127.50,1.0125,올라프
2,3,Galio,11.23,11.23.409.111,562.0,112,500.0,40.0,335,24,...,2466.0,1180.0,83.50,53.25,21.60,21.40,0,118.50,0.7844,갈리오
3,4,TwistedFate,11.23,11.23.409.111,534.0,94,333.0,39.0,330,21,...,2132.0,996.0,74.55,38.50,15.70,21.60,0,108.10,1.0074,트위스티드 페이트
4,5,XinZhao,11.23,11.23.409.111,570.0,92,274.0,55.0,345,35,...,2134.0,1209.0,94.50,53.25,19.90,14.91,0,114.00,1.0288,신 짜오
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,711,Vex,11.23,11.23.409.111,520.0,90,490.0,32.0,335,23,...,2050.0,1034.0,78.25,36.50,16.70,19.60,0,100.75,0.7827,벡스
153,777,Yone,11.23,11.23.409.111,550.0,85,500.0,0.0,345,28,...,1995.0,500.0,85.80,53.25,20.25,0.00,0,94.00,0.9969,요네
154,875,Sett,11.23,11.23.409.111,600.0,93,0.0,0.0,340,33,...,2181.0,0.0,101.00,53.25,15.50,0.00,0,128.00,0.8109,세트
155,876,Lillia,11.23,11.23.409.111,580.0,90,410.0,50.0,330,22,...,2110.0,1260.0,90.00,44.75,11.85,27.65,0,113.70,0.9119,릴리아


In [25]:
full_IRON_total = IRON_total.merge(champ_stat, how = 'left', on = 'name')
full_BRONZE_total = BRONZE_total.merge(champ_stat, how = 'left', on = 'name')
full_SILVER_total = SILVER_total.merge(champ_stat, how = 'left', on = 'name')
full_GOLD_total = GOLD_total.merge(champ_stat, how = 'left', on = 'name')
full_PLATINUM_total = PLATINUM_total.merge(champ_stat, how = 'left', on = 'name')
full_DIAMOND_total = DIAMOND_total.merge(champ_stat, how = 'left', on = 'name')
full_ABOVE_MONSTER_total = ABOVE_MASTER_total.merge(champ_stat, how = 'left', on = 'name')

In [26]:
full_IRON_total.to_csv('C:\\Users\\kkandoll\\ESports_Data_Analysis\\full_IRON_total.csv',index=False, encoding="utf-8-sig")
full_BRONZE_total.to_csv('C:\\Users\\kkandoll\\ESports_Data_Analysis\\full_BRONZE_total.csv',index=False, encoding="utf-8-sig")
full_SILVER_total.to_csv('C:\\Users\\kkandoll\\ESports_Data_Analysis\\full_SILVER_total.csv',index=False, encoding="utf-8-sig")
full_GOLD_total.to_csv('C:\\Users\\kkandoll\\ESports_Data_Analysis\\full_GOLD_total.csv',index=False, encoding="utf-8-sig")
full_PLATINUM_total.to_csv('C:\\Users\\kkandoll\\ESports_Data_Analysis\\full_PLATINUM_total.csv',index=False, encoding="utf-8-sig")
full_DIAMOND_total.to_csv('C:\\Users\\kkandoll\\ESports_Data_Analysis\\full_DIAMOND_total.csv',index=False, encoding="utf-8-sig")
full_ABOVE_MONSTER_total.to_csv('C:\\Users\\kkandoll\\ESports_Data_Analysis\\full_ABOVE_MONSTER_total.csv',index=False, encoding="utf-8-sig")

In [27]:
# heatmap_data = full_IRON_total[['wins', 'games', 'total_games', 'winRate',
#        'pickRate', 'championId', 'versionIndex',
#        'hp', 'hpperlevel', 'mp', 'mpperlevel', 'movespeed', 'armor',
#        'armorperlevel', 'spellblock', 'spellblockperlevel', 'attackrange',
#        'hpregen', 'hpregenperlevel', 'mpregen', 'mpregenperlevel', 'crit',
#        'critperlevel', 'attackdamage', 'attackdamageperlevel',
#        'attackspeedperlevel', 'attackspeed', 'hp_18', 'mp_18', 'armor_18',
#        'spellblock_18', 'hpregen_18', 'mpregen_18', 'crit_18',
#        'attackdamage_18', 'attackspeed_18']]

In [28]:
# colormap = plt.cm.PuBu
# plt.figure(figsize=(20, 20)) 
# plt.title("Person Correlation of Features", y = 1.05, size = 15) 
# sns.heatmap(heatmap_data.astype(float).corr(), linewidths = 0.1, vmax = 1.0, square = True, cmap = colormap, linecolor = "white", annot = True, annot_kws = {"size" : 10})